# Lab-07-1 Data Preprocessing

In [1]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

## Dataset

In [2]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [3]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

## Model

In [4]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(x_train.size(1), 3)
    def forward(self, x):
        return self.linear(x)

In [5]:
model = SoftmaxClassifier()
optimizer = optim.SGD(model.parameters(), lr=0.1)

## Train

In [6]:
def train(model, optimizer, x_train, y_train):
    n_epoch = 100
    for epoch in range(n_epoch):
        output = model(x_train)
        loss = F.cross_entropy(output, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print('Epoch {:4d}/{} Cost: {:.6f}'.format(
                epoch, n_epoch, loss.item()
            ))

In [7]:
train(model, optimizer, x_train, y_train)

Epoch    0/100 Cost: 1.255863
Epoch   10/100 Cost: 1.039199
Epoch   20/100 Cost: 0.971779
Epoch   30/100 Cost: 0.916874
Epoch   40/100 Cost: 0.871567
Epoch   50/100 Cost: 0.833773
Epoch   60/100 Cost: 0.801859
Epoch   70/100 Cost: 0.774575
Epoch   80/100 Cost: 0.750970
Epoch   90/100 Cost: 0.730325


## Test

In [8]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    loss = F.cross_entropy(prediction, y_test)
    
    print('Accuracy: {}% Cost: {:.6f}'.format(
        correct_count / len(y_test) * 100, loss.item()
    ))

In [9]:
test(model, optimizer, x_test, y_test)

Accuracy: 100.0% Cost: 0.193765


## ※ Data Preprocessing

In [10]:
x_train = torch.FloatTensor([[73, 80, 75], 
                             [93, 88, 93], 
                             [89, 91, 90], 
                             [96, 98, 100], 
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [11]:
mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu) / sigma
norm_x_train

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])

In [12]:
class MultivariateLinearRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

In [13]:
model = MultivariateLinearRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [14]:
n_epoch = 100
for epoch in range(n_epoch):
    output = model(norm_x_train)
    loss = F.mse_loss(output, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epoch, loss.item()
        ))

Epoch    0/100 Cost: 29706.806641
Epoch   10/100 Cost: 338.055420
Epoch   20/100 Cost: 4.280133
Epoch   30/100 Cost: 0.343428
Epoch   40/100 Cost: 0.237880
Epoch   50/100 Cost: 0.195791
Epoch   60/100 Cost: 0.167531
Epoch   70/100 Cost: 0.148329
Epoch   80/100 Cost: 0.135273
Epoch   90/100 Cost: 0.126404


In [15]:
x_test = torch.FloatTensor([[75, 66, 69]])
y_test = torch.FloatTensor([[143]])

In [16]:
norm_x_test = (x_test - mu) / sigma
norm_x_test

tensor([[-0.8865, -1.5197, -1.3152]])

In [17]:
prediction = model(norm_x_test)
prediction

tensor([[143.1215]], grad_fn=<AddmmBackward>)

In [18]:
y_test

tensor([[143.]])

In [19]:
F.mse_loss(prediction, y_test)

tensor(0.0148, grad_fn=<MseLossBackward>)